In [1]:
import os


In [3]:
pwd

'd:\\Nik\\UW-SoundSpeed'

In [2]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path
import pandas as pd


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from box.exceptions import BoxValueError
from uwsoundspeed.constants import *
from uwsoundspeed.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import shutil
import urllib.request as request
import zipfile
from uwsoundspeed import logger
from uwsoundspeed.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def handle_local_data(self):    
        if not os.path.exists(self.config.local_data_file):
            shutil.copy(self.config.source_URL, self.config.local_data_file)  # Assuming source_URL is the path to the local file
            logger.info(f"File copied to: {self.config.local_data_file}")
        else:
            logger.info(f"File already exists at destination: {self.config.local_data_file} of size: {get_size(Path(self.config.local_data_file))}")



    #in case if data need to download
    # def download_file(self):
    #     try:
    #         if not self.config.local_data_file.exists():
    #             filename, headers = request.urlretrieve(
    #                 url=self.config.source_URL,
    #                 filename=str(self.config.local_data_file)
    #             )
    #             logger.info(f"{filename} downloaded with following info: \n{headers}")
    #         else:
    #             logger.info(f"File already exists at: {self.config.local_data_file}")
    #     except Exception as e:
    #         logger.error(f"Failed to download file due to {e}")
    #         raise 


    
    def extract_zip_file(self):
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(self.config.unzip_dir)
            logger.info(f"Extracted zip file to {self.config.unzip_dir}")
        except Exception as e:
            logger.error(f"Failed to extract zip file due to {e}")
            raise

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.handle_local_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-18 13:29:38,827: INFO: common: YAML file: config\config.yaml loaded successfully]


[2024-03-18 13:29:38,843: INFO: common: YAML file: params.yaml loaded successfully]
[2024-03-18 13:29:38,845: INFO: common: Directory created at: artifacts]
[2024-03-18 13:29:38,847: INFO: common: Directory created at: artifacts/data_ingestion]
[2024-03-18 13:29:40,719: INFO: 1420110592: File copied to: artifacts/data_ingestion/data.zip]
[2024-03-18 13:30:00,170: INFO: 1420110592: Extracted zip file to artifacts/data_ingestion]
